In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchvision
from PIL import Image

In [ ]:
torch.cuda.is_available()

True

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
PATH="/content/gdrive/MyDrive"
os.listdir(PATH)

['Colab Notebooks', 'train_images', 'csv']

# Declaring Global Varibles

In [ ]:
CSV_PATH=PATH+"/csv"
IMAGE_PATH=PATH+"/train_images"
BS=8
LR=1e-2
WEIGHT_DECAY=5E-4
EPOCH=10

# Importing data

In [ ]:
data=pd.read_csv(CSV_PATH+"/train.csv")
data.head()

,image_name,label,template_name,xmin,ymin,xmax,ymax
0,0181 copie.jpg,0181 copie_0,0181 copie_0_0.jpg,216,696,545,1040
1,0181 copie.jpg,0181 copie_0_flipped,0181 copie_0_flipped_0.jpg,852,687,1210,1038
2,0181 copie.jpg,0181 copie_1,0181 copie_1_0.jpg,557,701,861,1080
3,0181 copie.jpg,0181 copie_2,0181 copie_2_0.jpg,194,1201,419,1380
4,0181 copie.jpg,0181 copie_2,0181 copie_2_1.jpg,1019,1201,1275,1379


In [ ]:
image_names=data['image_name'].unique()
print(f"We have {len(image_names)} images")

We have 18 images


In [ ]:
# Store the arrays of the image
image_names=list(data['image_name'].unique())
imageArray=pd.DataFrame({"image_name":[],"image_array":[]})
for im in image_names:
  img=Image.open(IMAGE_PATH+"/"+im)
  imageArray=pd.concat([imageArray,pd.DataFrame({"image_name":[im],"image_array":[np.array(img)]})])
  del img

In [ ]:
imageArray.head()

,image_name,image_array
0,0181 copie.jpg,"[[[145, 138, 132], [140, 133, 127], [140, 133,..."
0,0183 copie.jpg,"[[[153, 146, 140], [165, 158, 152], [133, 126,..."
0,0185 copie.jpg,"[[[163, 156, 148], [156, 149, 141], [151, 144,..."
0,0187 copie.jpg,"[[[151, 144, 138], [151, 144, 138], [148, 141,..."
0,0189 copie.jpg,"[[[143, 136, 130], [146, 139, 133], [133, 126,..."


# Pretext Task

    > Predict Scale

In [ ]:
# We won't save the arrays of the rescaled image, we will save the scaling factor and coordinates for crop
scaleData=pd.DataFrame({"image_name":[],"x_min":[],"y_min":[],"x_max":[],"y_max":[],"scaling_factor":[]})
scaleData["x_min"]=scaleData["image_name"].apply(lambda x:0)
scaleData["y_min"]=scaleData["image_name"].apply(lambda x:0)
scaleData["x_max"]=scaleData["image_name"].apply(lambda x:
                                                 imageArray[imageArray["image_name"]==x]["image_array"][0].shape[1])
scaleData["y_max"]=scaleData["image_name"].apply(lambda x:
                                                 imageArray[imageArray["image_name"]==x]["image_array"][0].shape[0])

In [ ]:
# We will randomly crop an image and scale it to 300 * 300 and store the scaling factor as a target variable
scaling_factors=np.arange(1,3.1,0.1)
def rescale_crop(num,image_name):
    """ Randomly crop an image and scale it to 300 * 300 """
    global scaleData
    global data
    global scaling_factors

    h,w=imageArray[imageArray['image_name']==image_name]["image_array"].values[0].shape[:2]

    # Choose randomly a scaling factor
    sc=np.random.choice(scaling_factors)

    # Randomly choose a height for image between 400 to 1000
    new_w=int(300*sc)
    new_h=int(300*sc)

    # Choose randomly the coordinates
    new_x=int(np.random.randint(0,w-2*new_w))
    new_y=int(np.random.randint(0,h-new_h))

    # image name
    name=image_name.split(".")[0]+"_scale_"+str(num)+".jpg"

    scaleData=pd.concat([scaleData,pd.DataFrame({"image_name":[name],"x_min":[new_x],
    "y_min":[new_y],"x_max":[new_x+new_w],"y_max":[new_y+new_h],"scaling_factor":[sc]})])
     


In [ ]:
%%time
for im in image_names:
    for i in range(200):
        rescale_crop(i,im)

CPU times: user 10.6 s, sys: 161 ms, total: 10.8 s
Wall time: 10.6 s


In [ ]:
le=LabelEncoder()
scaleData['scaling_factor']=le.fit_transform(scaleData['scaling_factor'])

In [ ]:
scaleData.head()

,image_name,x_min,y_min,x_max,y_max,scaling_factor
0,0181 copie_scale_0.jpg,434.0,2147.0,1214.0,2927.0,16
0,0181 copie_scale_1.jpg,597.0,1032.0,1257.0,1692.0,12
0,0181 copie_scale_2.jpg,363.0,2415.0,993.0,3045.0,11
0,0181 copie_scale_3.jpg,943.0,1584.0,1393.0,2034.0,5
0,0181 copie_scale_4.jpg,582.0,3090.0,1212.0,3720.0,11


In [ ]:

scaleData.to_csv(CSV_PATH+"/"+"scaleData.csv",index=None)

In [ ]:
imageArray[imageArray['image_name']=="0181 copie.jpg"]['image_array'].values[0].shape

(3806, 2072, 3)

In [ ]:
img=np.array(Image.open(IMAGE_PATH+"/"+"0181 copie.jpg"))
template=img[1206:1746,404:944,:]
template.shape

(540, 540, 3)

In [ ]:
class ScaleDataset(Dataset):
    def __init__(self, dataset, image_dataset, is_test=False, transform=None):
        #self.annotation_folder_path = csv_path
        self.dataset=dataset
        self.image_dataset=image_dataset
        self.all_images=self.dataset['image_name'].unique()
        self.transform = transform
        self.is_test = is_test
    
    def __getitem__(self,idx):
        img_name=self.all_images[idx]
        original_img_name=img_name.split("_")[0]+".jpg"
        coord=self.dataset[self.dataset['image_name']==img_name][["x_min","y_min","x_max","y_max"]].values[0]
        img=Image.fromarray(self.image_dataset[self.image_dataset['image_name']==original_img_name]['image_array'].values[0][
            int(coord[1]):int(coord[3]),int(coord[0]):int(coord[2])])
        #img=img.convert("RGB")

        if not self.is_test:
            annotations=self.dataset[self.dataset['image_name']==img_name]

            #self.box = self.get_xy(annotations)

            #self.new_box = torch.FloatTensor(self.box_resize(self.box, img))
            if self.transform is not None:
                img = self.transform(img)
            

            self.labels=torch.FloatTensor(annotations['scaling_factor'].values)

            return img, self.labels
        else:
            return img
    
    def __len__(self):
        return len(self.all_images)
        
    
    def collate_fn(self, batch):
        """
        :param batch: an iterable of N sets from __getitem__()
        :return: a tensor of images, lists of varying-size tensors of bounding boxes, labels, and difficulties
        """

        images = list()
        labels = list()
#         difficulties = list()

        for b in batch:
            images.append(b[0])
            labels.append(b[1])
#             difficulties.append(b[3])

        images = torch.stack(images, dim=0)

        return images, labels



In [ ]:
tsfm = transforms.Compose([
    transforms.Resize([300,300]),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [ ]:
x_train,x_test=train_test_split(scaleData,test_size=0.15,random_state=1)

In [ ]:
train_ds = ScaleDataset(x_train,imageArray,transform=tsfm)
train_dl = DataLoader(train_ds, batch_size=BS, shuffle=True, collate_fn=train_ds.collate_fn)

valid_ds = ScaleDataset(x_test,imageArray, transform=None)
valid_dl = DataLoader(valid_ds, batch_size=BS, shuffle=True, collate_fn=valid_ds.collate_fn)

In [ ]:
def decimate(tensor, m):
    """
    Decimate a tensor by a factor 'm', i.e. downsample by keeping every 'm'th value.
    This is used when we convert FC layers to equivalent Convolutional layers, BUT of a smaller size.
    :param tensor: tensor to be decimated
    :param m: list of decimation factors for each dimension of the tensor; None if not to be decimated along a dimension
    :return: decimated tensor
    """
    assert tensor.dim() == len(m)
    for d in range(tensor.dim()):
        if m[d] is not None:
            tensor = tensor.index_select(dim=d,
                                         index=torch.arange(start=0, end=tensor.size(d), step=m[d]).long())

    return tensor
class VGGBase(nn.Module):
    """
    VGG base convolutions to produce lower-level feature maps.
    """

    def __init__(self):
        super(VGGBase, self).__init__()

        # Standard convolutional layers in VGG16
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)  # stride = 1, by default
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)  # ceiling (not floor) here for even dims

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)  # retains size because stride is 1 (and padding)

        # Replacements for FC6 and FC7 in VGG16
        self.conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)  # atrous convolution

        self.conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
        # Load pretrained layers
        self.load_pretrained_layers()

        # Linear Layers
        self.linear1=nn.Linear(in_features=1024*19*19,out_features=512)
        self.linear2=nn.Linear(in_features=512,out_features=256)
        self.output=nn.Linear(in_features=256,out_features=21)


    def forward(self, image):
        """
        Forward propagation.
        :param image: images, a tensor of dimensions (N, 3, 300, 300)
        :return: lower-level feature maps conv4_3 and conv7
        """
        out = F.relu(self.conv1_1(image))  # (N, 64, 300, 300)
        out = F.relu(self.conv1_2(out))  # (N, 64, 300, 300)
        out = self.pool1(out)  # (N, 64, 150, 150)

        out = F.relu(self.conv2_1(out))  # (N, 128, 150, 150)
        out = F.relu(self.conv2_2(out))  # (N, 128, 150, 150)
        out = self.pool2(out)  # (N, 128, 75, 75)

        out = F.relu(self.conv3_1(out))  # (N, 256, 75, 75)
        out = F.relu(self.conv3_2(out))  # (N, 256, 75, 75)
        out = F.relu(self.conv3_3(out))  # (N, 256, 75, 75)
        out = self.pool3(out)  # (N, 256, 38, 38), it would have been 37 if not for ceil_mode = True

        out = F.relu(self.conv4_1(out))  # (N, 512, 38, 38)
        out = F.relu(self.conv4_2(out))  # (N, 512, 38, 38)
        out = F.relu(self.conv4_3(out))  # (N, 512, 38, 38)
        conv4_3_feats = out  # (N, 512, 38, 38)
        out = self.pool4(out)  # (N, 512, 19, 19)

        out = F.relu(self.conv5_1(out))  # (N, 512, 19, 19)
        out = F.relu(self.conv5_2(out))  # (N, 512, 19, 19)
        out = F.relu(self.conv5_3(out))  # (N, 512, 19, 19)
        out = self.pool5(out)  # (N, 512, 19, 19), pool5 does not reduce dimensions

        out = F.relu(self.conv6(out))  # (N, 1024, 19, 19)

        conv7_feats = F.relu(self.conv7(out))  # (N, 1024, 19, 19)

        # Linear Layers
        flattened_array=conv7_feats.reshape(-1,1024*19*19)
        out=F.relu(self.linear1(flattened_array))
        out=F.relu(self.linear2(out))
        output=F.softmax(self.output(out))

        # Lower-level feature maps
        return output

    def load_pretrained_layers(self):
        """
        As in the paper, we use a VGG-16 pretrained on the ImageNet task as the base network.
        There's one available in PyTorch, see https://pytorch.org/docs/stable/torchvision/models.html#torchvision.models.vgg16
        We copy these parameters into our network. It's straightforward for conv1 to conv5.
        However, the original VGG-16 does not contain the conv6 and con7 layers.
        Therefore, we convert fc6 and fc7 into convolutional layers, and subsample by decimation. See 'decimate' in utils.py.
        """
        # Current state of base
        state_dict = self.state_dict()
        param_names = list(state_dict.keys())

        # Pretrained VGG base
        pretrained_state_dict = torchvision.models.vgg16(pretrained=True).state_dict()
        pretrained_param_names = list(pretrained_state_dict.keys())

        # Transfer conv. parameters from pretrained model to current model
        for i, param in enumerate(param_names[:-4]):  # excluding conv6 and conv7 parameters
            state_dict[param] = pretrained_state_dict[pretrained_param_names[i]]

        # Convert fc6, fc7 to convolutional layers, and subsample (by decimation) to sizes of conv6 and conv7
        # fc6
        conv_fc6_weight = pretrained_state_dict['classifier.0.weight'].view(4096, 512, 7, 7)  # (4096, 512, 7, 7)
        conv_fc6_bias = pretrained_state_dict['classifier.0.bias']  # (4096)
        state_dict['conv6.weight'] = decimate(conv_fc6_weight, m=[4, None, 3, 3])  # (1024, 512, 3, 3)
        state_dict['conv6.bias'] = decimate(conv_fc6_bias, m=[4])  # (1024)
        # fc7
        conv_fc7_weight = pretrained_state_dict['classifier.3.weight'].view(4096, 4096, 1, 1)  # (4096, 4096, 1, 1)
        conv_fc7_bias = pretrained_state_dict['classifier.3.bias']  # (4096)
        state_dict['conv7.weight'] = decimate(conv_fc7_weight, m=[4, 4, None, None])  # (1024, 1024, 1, 1)
        state_dict['conv7.bias'] = decimate(conv_fc7_bias, m=[4])  # (1024)

        # Note: an FC layer of size (K) operating on a flattened version (C*H*W) of a 2D image of size (C, H, W)...
        # ...is equivalent to a convolutional layer with kernel size (H, W), input channels C, output channels K...
        # ...operating on the 2D image of size (C, H, W) without padding

        self.load_state_dict(state_dict)

        print("\nLoaded base model.\n")

In [ ]:
vgg=VGGBase().to("cuda:0")


Loaded base model.



In [ ]:
summary(vgg,(3,300,300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 300, 300]           1,792
            Conv2d-2         [-1, 64, 300, 300]          36,928
         MaxPool2d-3         [-1, 64, 150, 150]               0
            Conv2d-4        [-1, 128, 150, 150]          73,856
            Conv2d-5        [-1, 128, 150, 150]         147,584
         MaxPool2d-6          [-1, 128, 75, 75]               0
            Conv2d-7          [-1, 256, 75, 75]         295,168
            Conv2d-8          [-1, 256, 75, 75]         590,080
            Conv2d-9          [-1, 256, 75, 75]         590,080
        MaxPool2d-10          [-1, 256, 38, 38]               0
           Conv2d-11          [-1, 512, 38, 38]       1,180,160
           Conv2d-12          [-1, 512, 38, 38]       2,359,808
           Conv2d-13          [-1, 512, 38, 38]       2,359,808
        MaxPool2d-14          [-1, 512,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(vgg.parameters(),lr=LR,weight_decay=WEIGHT_DECAY)

In [ ]:
%%time
for epoch in range(EPOCH):
    vgg.train()
    train_loss=[]
    for step,(img,labels) in enumerate(train_dl):
        labels=torch.tensor(labels)
        labels=labels.long()
        labels=labels.to("cuda:0")
        img=img.to("cuda:0")
        pred=vgg(img)
        loss=criterion(pred,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
    
    print(f"Epoch {epoch+1}, train loss: {np.mean(train_loss)}")



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1, train loss: 3.044392997228757
Epoch 2, train loss: 3.044129522288748
Epoch 3, train loss: 3.0437842816036613
Epoch 4, train loss: 3.04317569732666
Epoch 5, train loss: 3.041479242065868
Epoch 6, train loss: 3.0227061363797896
Epoch 7, train loss: 3.0169280278776083
Epoch 8, train loss: 2.993815061006471
Epoch 9, train loss: 2.979552818029419
Epoch 10, train loss: 2.961246898217861
CPU times: user 1h 22min 47s, sys: 9.59 s, total: 1h 22min 57s
Wall time: 1h 22min 51s
